#2024

# Introduction to Pytorch & Neural Networks

Chapter Learning Objectives
Imports
1. Introduction
2. PyTorch's Tensor
3. Neural Network Basics
4. Activation Functions
5. Neural Network Classification


## Learning Objectives
Describe the difference between NumPy and torch arrays (np.array vs. torch.Tensor).

Explain fundamental concepts of neural networks such as layers, nodes, activation functions, etc.

Create a simple neural network in PyTorch for regression or classification.



<a name='1'></a>
## 1 - Imports

As usual, begin by loading in the packages.

In [3]:
import sys
import NumPy as np
import pandas as pd
import torch
from torchsummary import summary
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import make_regression, make_circles, make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from utils.plotting import *

ModuleNotFoundError: No module named 'NumPy'

<a name='1-1'></a>
### 1.1 - Introduction

PyTorch is a Python-based tool for scientific computing that provides several main features:

torch.Tensor, an n-dimensional array similar to that of NumPy, but which can run on GPUs

Computational graphs and an automatic differentiation enginge for building and training neural networks

You can install PyTorch from: https://pytorch.org/.



<a name='1-1'></a>
### 2 - PyTorch’s Tensor

PIn PyTorch a tensor is just like NumPy’s ndarray which most readers will be familiar with already (if not, check out Chapter 5 and Chapter 6 of my Python Programming for Data Science course).

A key difference between PyTorch’s torch.Tensor and NumPy’s np.array is that torch.Tensor was constructed to integrate with GPUs and PyTorch’s computational graphs (more on that next chapter though).



<a name='1-1'></a>
### 2.1. - ndarray vs tensor
Creating and working with tensors is much the same as with NumPy ndarrays. You can create a tensor with torch.tensor():


In [2]:
tensor_1 = torch.tensor([1, 2, 3])
tensor_2 = torch.tensor([1, 2, 3], dtype=torch.float32)
tensor_3 = torch.tensor(np.array([1, 2, 3]))

for t in [tensor_1, tensor_2, tensor_3]:
    print(f"{t}, dtype: {t.dtype}")

NameError: name 'torch' is not defined

PyTorch also comes with most of the NumPy functions you’re probably already familiar with:

In [ ]:
torch.zeros(2, 2)  # zeroes

In [ ]:
torch.ones(2, 2)  # ones

In [ ]:
torch.ones(2, 2)  # ones

In [ ]:
torch.randn(3, 2)  # random normal

In [ ]:
torch.rand(2, 3, 2)  # rand uniform

ust like in NumPy we can look at the shape of a tensor with the .shape attribute:

In [4]:
x = torch.rand(2, 3, 2, 2)
x.shape

NameError: name 'torch' is not defined

In [ ]:
x.ndim

<a name='2'></a>
## 2.2. Tensors and Data Types

Different data types have different memory and computational implications (see Chapter 6 of Python Programming for Data Science for more). In Pytorch we’ll be building networks that require thousands or even millions of floating point calculations! In such cases, using a smaller dtype like float32 can significantly speed up computations and reduce memory requirements. The default float dtype in pytorch float32, as opposed to NumPy’s float64. In fact some operations in Pytorch will even throw an error if you pass a high-memory dtype!

In [ ]:
print(np.array([3.14159]).dtype)
print(torch.tensor([3.14159]).dtype)

But just like in NumPy, you can always specify the particular dtype you want using the dtype argument:

In [ ]:
print(torch.tensor([3.14159], dtype=torch.float64).dtype)

<a name='2.3'></a>
## 2.3. Operations on Tensors

Tensors operate just like ndarrays and have a variety of familiar methods that can be called off them:
 

In [ ]:
a = torch.rand(1, 3)
b = torch.rand(3, 1)

a + b  # broadcasting betweean a 1 x 3 and 3 x 1 tensor

In [ ]:
a * b

In [ ]:
a.mean()

In [ ]:
a.sum()

<a name='2.4'></a>
### 2.4. Indexing

UOnce again, same as NumPy!

In [ ]:
X = torch.rand(5, 2)
print(X)

In [ ]:
print(X[0, :])
print(X[0])
print(X[:, 0])

<a name='2.5'></a>
## 2.5. NumPy Bridge
Sometimes we might want to convert a tensor back to a NumPy array. We can do that using the .numpy() method:

In [5]:
X = torch.rand(3,3)
print(type(X))
X_NumPy = X.NumPy()
print(type(X_NumPy))

NameError: name 'torch' is not defined

<a name='2.6'></a>
## 2.6. GPU and CUDA Tensors

GPU stands for “graphical processing unit” (as opposed to a CPU: central processing unit). GPUs were originally developed for gaming, they are very fast at performing operations on large amounts of data by performing them in parallel (think about updating the value of all pixels on a screen very quickly as a player moves around in a game). More recently, GPUs have been adapted for more general purpose programming. Neural networks can typically be broken into smaller computations that can be performed in parallel on a GPU. PyTorch is tightly integrated with CUDA - a software layer that facilitates interactions with a GPU (if you have one). You can check if you have GPU capability using:

In [ ]:
torch.cuda.is_available()  # my MacBook Pro does not have a GPU

When training on a machine that has a GPU, you need to tell PyTorch you want to use it. You’ll see the following at the top of most PyTorch code:

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

You can then use the device argument when creating tensors to specify whether you wish to use a CPU or GPU. Or if you want to move a tensor between the CPU and GPU, you can use the .to() method:

In [ ]:
X = torch.rand(2, 2, 2, device=device)
print(X.device)

In [ ]:
# X.to('cuda')  # this would give me an error as I don't have a GPU so I'm commenting out

We’ll revisit GPUs later in the course when we are working with bigger datasets and more complex networks. For now, we can work on the CPU just fine

<a name='3'></a>
## 3. Neural Network Basics

It’s probably that you’ve already learned about several machine learning algorithms (kNN, Random Forest, SVM, etc.). Neural networks are simply another algorithm and actually one of the simplest in my opinion! As we’ll see, a neural network is just a sequence of linear and non-linear transformations. Often you see something like this when learning about/using neural networks:

So what on Earth does that all mean? Well we are going to build up some intuition one step at a time.



##3.1. Simple Linear Regression with a Neural Network
Let’s create a simple regression dataset with 500 observations:

In [ ]:
X, y = make_regression(n_samples=500, n_features=1, random_state=0, noise=10.0)
plot_regression(X, y)

In [ ]:
sk_model = LinearRegression().fit(X, y)
plot_regression(X, y, sk_model.predict(X))

In [ ]:
print(f"w_0: {sk_model.intercept_:.2f} (bias/intercept)")
print(f"w_1: {sk_model.coef_[0]:.2f}")

#https://www.tomasbeuzen.com/deep-learning-with-pytorch/chapters/chapter3_pytorch-neural-networks-pt1.html#